<a href="https://colab.research.google.com/github/beckynevin/Kinematic_ML/blob/master/PCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Apparently you can clone repos into collab:

In [1]:
!git clone https://github.com/salesforce/PCL.git

Cloning into 'PCL'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 141 (delta 46), reused 134 (delta 42), pack-reused 0
Receiving objects: 100% (141/141), 191.83 KiB | 1.00 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
# This is required for the above to work:
!pip install faiss-gpu


     |████████████████████████████████| 89.4MB 36kB/s 


Try to get useful information with (this is wrong but I'd like to be able to do this eventually):
parser(PCL/main_pcl.py)

In [ ]:
#!rm -r pngs/
#!rm -r _MACOSX/
#!rm -r Galaxies/train/

rm: cannot remove 'pngs/': No such file or directory
rm: cannot remove '_MACOSX/': No such file or directory


In [4]:

!unzip train.zip
!unzip test.zip
# default input is folders for each galaxy 

Archive:  train.zip
replace train/8464-12701/180.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/8262-12705/232.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/7990-12702/675.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/9185-12704/628.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/9863-12703/143.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/8452-12702/306.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/8262-12702/174.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/8554-12703/487.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  test.zip
replace test/8323-12705/228.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
!python PCL/main_pcl.py -a resnet18 --lr 0.03 --workers 10\
  --batch-size 128 --temperature 0.2 \
  --mlp --aug-plus --cos  --dist-url 'tcp://localhost:10001' \
  --multiprocessing-distributed --world-size 1 \
  --rank 0 --num-cluster 5 --exp-dir PCL/experiment_PCL/ \
   .

# batch-size = mini-batch size (total batch size of all GPUs on the current node)
# multiprocessing-distributed = use multiprocessing training to launch N processes per node, which has
# N GPUs
# num-cluster = number of clusters (must specify or it will default to a million)

# Getting an error about amount of memory avail to CUDA, apparently you need to 
# reduce the mini batch size: https://github.com/pytorch/pytorch/issues/16417

Use GPU: 0 for training
=> creating model 'resnet18'
MoCo(
  (encoder_q): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d

In [ ]:
# Now you need your data (optional if you want mnist)
import keras
import numpy as np
import torch
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

print(type(train_images), np.shape(train_images))
torch.save(train_images, 'PCL/train_images/images')

# Okay try instead getting the whole repo for mnist in .png format
!rm -r mnist_png/
!git clone https://github.com/myleott/mnist_png.git

!tar -xf mnist_png/mnist_png.tar.gz

!python PCL/main_pcl.py -a resnet50 --lr 0.03 --workers 10\
  --batch-size 128 --temperature 0.2 \
  --mlp --aug-plus --cos --dist-url 'tcp://localhost:10001' \
  --multiprocessing-distributed --world-size 1 \
  --rank 0 --exp-dir PCL/experiment_PCL/ mnist_png/

# batch-size = mini-batch size (total batch size of all GPUs on the current node)
# multiprocessing-distributed = use multiprocessing training to launch N processes per node, which has
# N GPUs
# num-cluster = number of clusters

# Getting an error about amount of memory avail to CUDA, apparently you need to 
# reduce the mini batch size: https://github.com/pytorch/pytorch/issues/16417

In [ ]:
import torch.cuda

torch.cuda.is_available()


SyntaxError: ignored